# 包 `tqdm`

`tqdm` 是Python中一个进度条库，可以为任何可迭代对象包装一个智能进度条。

`tqdm` 本质上是一个类，其构造函数包含众多参数，包括：
- `iterable`: 可迭代对象（如列表、生成器等）
- `desc`: 字符串，进度条前的描述文本
- `total`: 预期的迭代总数，默认为None
- `leave`: 布尔值，是否保留进度条的最终状态（默认True）
- `ncols`: 整数，进度条的显示宽度
- `mininterval`: 浮点数，最小更新间隔（秒），默认0.1秒
- `maxinterval`: 浮点数，最大更新间隔（秒），默认10秒
- `ascii`: 布尔值，是否使用ASCII字符显示
- `disable`: 布尔值，是否禁用进度条显示
- `unit`: 字符串，进度单位，默认为'it'（iterations）

例子：
```python
from tqdm import tqdm
import time

for i in tqdm(range(100), 
              desc="处理中", 
              unit="文件", 
              unit_scale=True):
    time.sleep(0.01)
```